In [13]:
from qiskit.circuit.library import EfficientSU2
import re
from qiskit import QuantumCircuit

parametrised_gates = [
    "rx",
    "ry",
    "rz",
    "crx",
    "cry",
    "crz",
    "cp",
    "rxx",
    "ryy",
    "rzz",
]

def get_circuits(circuit_chromosomes):
    circuits = []
    
    for circuit_chromosome in circuit_chromosomes:
        # Initialise circuit
        circuit = QuantumCircuit(len(circuit_chromosome[0]))

        # Gate map for Qiskit Aer native gates with explanations
        chromosome_qiskit_gate_map = {
            "w": lambda qubit: circuit.barrier(qubit),  # Barrier (used for blank "wires")
            "-": None,  # Placeholder for control qubits (no operation)
            "x": lambda qubit: circuit.x(qubit),  # Pauli-X (NOT) gate
            "y": lambda qubit: circuit.y(qubit),  # Pauli-Y gate
            "z": lambda qubit: circuit.z(qubit),  # Pauli-Z gate
            "h": lambda qubit: circuit.h(qubit),  # Hadamard gate
            "s": lambda qubit: circuit.s(qubit),  # S (Phase) gate: R_z(π/2)
            "sdg": lambda qubit: circuit.sdg(qubit),  # S-dagger (Inverse Phase) gate: R_z(-π/2)
            "t": lambda qubit: circuit.t(qubit),  # T gate: R_z(π/4)
            "tdg": lambda qubit: circuit.tdg(qubit),  # T-dagger gate: R_z(-π/4)
            "rx": lambda qubit, theta: circuit.rx(theta, qubit),  # Rotation around the X axis: R_x(θ)
            "ry": lambda qubit, theta: circuit.ry(theta, qubit),  # Rotation around the Y axis: R_y(θ)
            "rz": lambda qubit, theta: circuit.rz(theta, qubit),  # Rotation around the Z axis: R_z(θ)
            "cx": lambda control_qubit, target_qubit: circuit.cx(control_qubit, target_qubit),  # CNOT (Controlled-X) gate
            "cy": lambda control_qubit, target_qubit: circuit.cy(control_qubit, target_qubit),  # Controlled-Y gate
            "cz": lambda control_qubit, target_qubit: circuit.cz(control_qubit, target_qubit),  # Controlled-Z gate
            "swap": lambda q1, q2: circuit.swap(q1, q2),  # SWAP gate (exchange qubits)
            "ccx": lambda q1, q2, target_qubit: circuit.ccx(q1, q2, target_qubit),  # Toffoli gate (Controlled-Controlled-X)
            "cswap": lambda control_qubit, q1, q2: circuit.cswap(control_qubit, q1, q2),  # Controlled-SWAP gate
            "crx": lambda control_qubit, target_qubit, theta: circuit.crx(theta, control_qubit, target_qubit),  # Controlled-RX rotation gate
            "cry": lambda control_qubit, target_qubit, theta: circuit.cry(theta, control_qubit, target_qubit),  # Controlled-RY rotation gate
            "crz": lambda control_qubit, target_qubit, theta: circuit.crz(theta, control_qubit, target_qubit),  # Controlled-RZ rotation gate
            "cp": lambda control_qubit, target_qubit, theta: circuit.cp(theta, control_qubit, target_qubit),  # Controlled-Phase gate
            "rxx": lambda q1, q2, theta: circuit.rxx(theta, q1, q2),  # Ising interaction: R_xx(θ) (rotation on the XX interaction)
            "ryy": lambda q1, q2, theta: circuit.ryy(theta, q1, q2),  # Ising interaction: R_yy(θ) (rotation on the YY interaction)
            "rzz": lambda q1, q2, theta: circuit.rzz(theta, q1, q2),  # Ising interaction: R_zz(θ) (rotation on the ZZ interaction)
        }

        # Helper to apply gates
        for block in circuit_chromosome:
            for qubit in range(len(block)):
                gate_spec = block[qubit]
                if gate_spec == "-":
                    continue
                elif "(" in gate_spec:
                    gate, args = re.match(r"(\w+)\((.+)\)", gate_spec).groups()
                    if gate in parametrised_gates:
                        args = list(args.split(","))
                        args[-1] = float(args[-1])
                        args[:-1] = map(int, args[:-1])
                    else:
                        args = list(map(int, args.split(",")))
                    chromosome_qiskit_gate_map[gate](*args)
                else:
                    chromosome_qiskit_gate_map[gate_spec](qubit)
        circuit.save_statevector()

        circuits.append(circuit.copy())

    return circuits

qft_chromosome = [
        ["h(0)", "w", "w"],  # Hadamard on qubit 0
        ["cp(0,1,1.5707963267948966)", "w", "w"],  # Controlled rotation π/2 between qubits 0 and 1
        ["cp(0,2,0.7853981633974483)", "w", "w"],  # Controlled rotation π/4 between qubits 0 and 2
        ["w", "h(1)", "w"],  # Hadamard on qubit 1
        ["w", "cp(1,2,1.5707963267948966)", "w"],  # Controlled rotation π/2 between qubits 1 and 2
        ["w", "w", "h(2)"],  # Hadamard on qubit 2
        ["swap(0,2)", "w", "w"],  # Swap qubits 0 and 2
    ]
    
circuit = get_circuits([qft_chromosome])[0]
print(circuit)

 
#n_qubits = 8
#circuit = EfficientSU2(n_qubits)
#circuit.decompose().draw("mpl")

     ┌───┐                      ░   ░      ░                   statevector 
q_0: ┤ H ├─■───────────■────────░───░──────░─────────────X──────────░──────
     └─░─┘ │P(π/2)  ░  │        ░ ┌───┐    ░     ░   ░   │          ░      
q_1: ──░───■────────░──┼────────░─┤ H ├─■────────░───░───┼──────────░──────
       ░      ░     ░  │P(π/4)  ░ └─░─┘ │P(π/2)  ░ ┌───┐ │  ░       ░      
q_2: ──░──────░────────■────────░───░───■────────░─┤ H ├─X──░───────░──────
       ░      ░                 ░   ░            ░ └───┘    ░       ░      


In [16]:
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import EstimatorV2 as Estimator
 
observable = SparsePauliOp("Z" * 3)
params = [0.1] * circuit.num_parameters
 
exact_estimator = Estimator()
# The circuit needs to be transpiled to the AerSimulator target
pass_manager = generate_preset_pass_manager(3, AerSimulator())
isa_circuit = pass_manager.run(circuit)
pub = (isa_circuit, observable, params)
job = exact_estimator.run([pub])
result = job.result()
pub_result = result[0]
exact_value = float(pub_result.data.evs)
exact_value

0.0

In [15]:
from qiskit_aer.noise import NoiseModel, depolarizing_error
 
noise_model = NoiseModel()
cx_depolarizing_prob = 0.02
noise_model.add_all_qubit_quantum_error(
    depolarizing_error(cx_depolarizing_prob, 2), ["cx"]
)
 
noisy_estimator = Estimator(
    options=dict(backend_options=dict(noise_model=noise_model))
)
job = noisy_estimator.run([pub])
result = job.result()
pub_result = result[0]
noisy_value = float(pub_result.data.evs)
noisy_value

0.0

# Atempt 2

In [30]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error

In [31]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Replace 'YOUR_API_TOKEN' with your actual API token
QiskitRuntimeService.save_account(channel='ibm_quantum', token='245e9b74ee58024ea6194faa8365e9ee507ccff553f3f7d97a2bf8b34f9f2fe6f32a75b17cdada69579fae02e328c0d283b8f727cd95455a03243c376c03b076', overwrite=True)

In [32]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Load IBM Quantum account
service = QiskitRuntimeService()

In [34]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Load IBM Quantum account
service = QiskitRuntimeService()

# List available backends
backends = service.backends()
print("Available backends:")
for backend in backends:
    print(backend.name)


Available backends:
ibm_brisbane
ibm_kyiv
ibm_sherbrooke


In [35]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Load IBM Quantum account
service = QiskitRuntimeService()

# Select a backend
backend = service.backend('ibm_brisbane')

# Create a noise model from the backend
noise_model = NoiseModel.from_backend(backend)


In [36]:
simulator = AerSimulator(noise_model=noise_model)


In [37]:
result = simulator.run(circuit).result()

In [38]:
counts = result.get_counts()
print(counts)

{'000': 0.125, '001': 0.125, '010': 0.125, '011': 0.125, '100': 0.125, '101': 0.125, '110': 0.125, '111': 0.125}
